In [16]:
# (KR) - (2026.1.29)
# 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(temperature=0.1)

examples = [
    {
        "title": "The Godfather",
        "answer": """
        The Godfather => 👨‍👨‍👦🔫🍝
        """,
    },
    {
        "title": "Top Gun",
        "answer": """
        Top Gun =>  🛩️👨‍✈️🔥
        """,
    },
    {
        "title": "Parasite",
        "answer": """
        Parasite => 🏠🕶️🍑
        """,
    },
]

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=300,
    return_messages=True,
    memory_key="history"
)

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Pls give me 3 icons explaining movie about {title}?"),
        ("ai","{answer}")
    ]
) 
    

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a movie expert. you MUST ONLY provide exactly 3 icons for any movie. Not allow to provide more or less than 3. "),
        few_shot_prompt, 
        MessagesPlaceholder(variable_name="history"), 
        ("human","What is 3 movie icons about {title}?"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(title):
    result = chain.invoke({"title":title})
    memory.save_context({"input":title},{"output":result.content})
    print(result)

invoke_chain("Apocalypse Now")
invoke_chain("Dead Poets Society")


content='\n        Apocalypse Now => 🚁🌴🔥'
content='Dead Poets Society => 📚🍎🎭'


In [17]:
invoke_chain("What is the movie I've just asked ?")

content='Dead Poets Society => 📚🍎🎭'
